## Data parsing/preprocessing

In [ ]:
important_fields = [
 'activity_id',
 'assay_chembl_id',
 'bao_endpoint',
 'bao_format',
 'bao_label',
 'canonical_smiles',
 'molecule_chembl_id',
 'parent_molecule_chembl_id',
 'pchembl_value',
 'potential_duplicate',
 'relation',
 'standard_flag',
 'standard_relation',
 'standard_text_value',
 'standard_type',
 'standard_units',
 'standard_upper_value',
 'standard_value',
 'target_chembl_id',
 'target_organism',
 'target_pref_name',
 'target_tax_id',
]

In [ ]:
!mkdir /content/logs

In [ ]:
import logging
logging.basicConfig(
    filename="/content/logs/activities_chembl_parse.log",
    level=logging.INFO,
    force = True,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

In [ ]:
import requests
import time
from pprint import pprint
from random import uniform


headers = {'Accept': 'application/json'}
target_id = "CHEMBL1907600"

limit = 100



def extract_features(activities):
  extract = lambda x: {key:x[key] for key in important_fields}
  objs = []
  for act in activities:
    objs.append(extract(act))
  return objs


def parse_activities_data(target_id, limit, headers):
  rows = []
  offset = 0
  logging.info('...Start...')
  while True:
    try:
      url = f'https://www.ebi.ac.uk/chembl/api/data/activity?target_chembl_id={target_id}&limit={limit}&offset={offset}'
      req = requests.get(url, headers=headers)
      if not req.json()['activities']:
        break
      rows.extend(extract_features(req.json()['activities']))
      logging.info(f'parsed {len(rows)} objects')
      offset += limit
    except Exception as ex:
      logging.info(f'parsing error: request code - {req.status_code}, exception: {ex.message}')

  return rows

rows = parse_activities_data(target_id, limit, headers)

In [ ]:
len(rows)

4010

In [ ]:
!mkdir /content/drive/MyDrive/datacon2025/main/nlrp

In [ ]:
df_dict = {key: [] for key in rows[0].keys()}
for obj in rows:
  for key in df_dict.keys():
    df_dict[key].append(obj[key])

In [ ]:
import pandas as pd
df = pd.DataFrame(df_dict)

In [ ]:
df.head()

,activity_id,assay_chembl_id,bao_endpoint,bao_format,bao_label,canonical_smiles,molecule_chembl_id,parent_molecule_chembl_id,pchembl_value,potential_duplicate,...,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id
0,72750,CHEMBL663602,BAO_0000190,BAO_0000223,protein complex format,CC(=O)O/N=C1/C(c2c(O)[nH]c3cc(Cl)ccc23)=Nc2ccc...,CHEMBL174068,CHEMBL174068,6.70,0,...,=,None,IC50,nM,None,200.0,CHEMBL1907600,Homo sapiens,Cyclin-dependent kinase 5/CDK5 activator 1,9606
1,73782,CHEMBL663602,BAO_0000190,BAO_0000223,protein complex format,Cn1c(O)c(C2=Nc3ccccc3C2=O)c2ccccc21,CHEMBL435707,CHEMBL435707,None,0,...,>,None,IC50,nM,None,100000.0,CHEMBL1907600,Homo sapiens,Cyclin-dependent kinase 5/CDK5 activator 1,9606
2,74885,CHEMBL663602,BAO_0000190,BAO_0000223,protein complex format,O=C1C(c2c(O)[nH]c3cc(Br)ccc23)=Nc2ccccc21,CHEMBL176904,CHEMBL176904,4.28,0,...,=,None,IC50,nM,None,53000.0,CHEMBL1907600,Homo sapiens,Cyclin-dependent kinase 5/CDK5 activator 1,9606
3,74888,CHEMBL663602,BAO_0000190,BAO_0000223,protein complex format,O=Nc1c(-c2c(O)[nH]c3cc(Cl)ccc23)[nH]c2ccccc12,CHEMBL173040,CHEMBL173040,7.00,0,...,=,None,IC50,nM,None,100.0,CHEMBL1907600,Homo sapiens,Cyclin-dependent kinase 5/CDK5 activator 1,9606
4,78255,CHEMBL663602,BAO_0000190,BAO_0000223,protein complex format,CC(=O)O/N=C1/C(c2c(O)[nH]c3ccccc23)=Nc2ccccc21,CHEMBL176896,CHEMBL176896,6.16,0,...,=,None,IC50,nM,None,700.0,CHEMBL1907600,Homo sapiens,Cyclin-dependent kinase 5/CDK5 activator 1,9606


In [ ]:
df.to_csv('/content/drive/MyDrive/datacon2025/main/nlrp/nlrp_raw_df.csvb')

In [ ]:
df['standard_type'].value_counts()

,count
standard_type,
IC50,2447
Residual Activity,632
Inhibition,407
Activity,298
AC50,119
Ki,50
Residual activity,18
% of inhibition,9
% of control,8


In [ ]:
ic50_df = df[df['standard_type'] == 'IC50']
ic50_df.shape

(2447, 22)

In [ ]:
ic50_df['target_organism'].value_counts()

,count
target_organism,
Homo sapiens,2447


In [ ]:
ic50_df = ic50_df.drop(['target_organism'], axis = 1)


In [ ]:
ic50_df['target_chembl_id'].value_counts()

,count
target_chembl_id,
CHEMBL1907600,2447


In [ ]:
ic50_df = ic50_df.drop(['target_chembl_id', 'target_pref_name', 'target_tax_id'], axis = 1)

In [ ]:
ic50_df['standard_relation'].value_counts()

,count
standard_relation,
=,1445
>,924
<,21
>=,9


In [ ]:
ic50_df = ic50_df[ic50_df['standard_relation'] == '=']


In [ ]:
ic50_df = ic50_df.drop(['relation', 'standard_relation'], axis = 1)


In [ ]:
ic50_df['bao_label'].value_counts()

,count
bao_label,
protein complex format,1156
assay format,174
cell-based format,106
single protein format,9


In [ ]:
ic50_df = ic50_df[ic50_df['bao_label'] == 'protein complex format']
ic50_df.shape

(1156, 16)

In [ ]:
ic50_df = ic50_df.drop(['bao_label', 'bao_format', 'bao_endpoint'], axis = 1)
ic50_df.shape

(1156, 13)

In [ ]:
ic50_df['standard_units'].value_counts()

,count
standard_units,
nM,1156


In [ ]:
ic50_df = ic50_df.drop(['standard_upper_value', 'standard_text_value', 'standard_flag'], axis = 1)
ic50_df.columns

Index(['activity_id', 'assay_chembl_id', 'canonical_smiles',
       'molecule_chembl_id', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'standard_type', 'standard_units',
       'standard_value'],
      dtype='object')

In [ ]:
ic50_df['molecule_chembl_id'].value_counts()

,count
molecule_chembl_id,
CHEMBL14762,13
CHEMBL388978,7
CHEMBL1276127,6
CHEMBL296586,5
CHEMBL409450,4
...,...
CHEMBL400433,1
CHEMBL239709,1
CHEMBL392228,1


In [ ]:
ic50_df['standard_value'] = ic50_df['standard_value'].astype(float)
ic50_df['pchembl_value'] = ic50_df['pchembl_value'].astype(float)

In [ ]:
final_df = ic50_df.groupby('molecule_chembl_id').agg({
    'canonical_smiles': 'first',
    'standard_value': 'median',
    'pchembl_value': 'median'
})

final_df

,canonical_smiles,standard_value,pchembl_value
molecule_chembl_id,,,
CHEMBL100309,O=C1Cc2c([nH]c3ccc(C(F)(F)F)cc23)-c2cc(Br)ccc2N1,2999.16,5.52
CHEMBL100312,CC(C)(C)OC(=O)n1c2c(c3cc(Br)ccc31)CC(=O)Nc1ccc...,799834.26,NaN
CHEMBL1092509,O=C1Nc2c(Br)cccc2/C1=C1/Nc2ccccc2/C1=N\O,33000.00,4.48
CHEMBL1094304,CC[C@H](CO)Nc1nc(NCc2ccccc2O)c2ncn(C(C)C)c2n1,270.00,6.57
CHEMBL1094408,COc1ccc(N2CCN(C)CC2)cc1Nc1ncc2c(n1)-c1c(c(C(N)...,219.00,6.66
...,...,...,...
CHEMBL98554,O=C1Cc2c([nH]c3cc(Br)ccc23)-c2ccccc2N1,2697.74,5.57
CHEMBL99037,O=C1Cc2c([nH]c3ccc(F)cc23)-c2ccccc2N1,1300.17,5.89
CHEMBL99423,COc1cccc2c1NC(=O)Cc1c-2[nH]c2ccccc12,1000000.00,NaN


In [ ]:
import numpy as np

def pchembl_from_nM(nM_value):
    molar = nM_value * 1e-9
    return -np.log10(molar)

In [ ]:
final_df['log_ic50(pchembl)'] = pchembl_from_nM(final_df['standard_value'])
final_df

,canonical_smiles,standard_value,pchembl_value,log_ic50(pchembl)
molecule_chembl_id,,,,
CHEMBL100309,O=C1Cc2c([nH]c3ccc(C(F)(F)F)cc23)-c2cc(Br)ccc2N1,2999.16,5.52,5.523000
CHEMBL100312,CC(C)(C)OC(=O)n1c2c(c3cc(Br)ccc31)CC(=O)Nc1ccc...,799834.26,NaN,3.097000
CHEMBL1092509,O=C1Nc2c(Br)cccc2/C1=C1/Nc2ccccc2/C1=N\O,33000.00,4.48,4.481486
CHEMBL1094304,CC[C@H](CO)Nc1nc(NCc2ccccc2O)c2ncn(C(C)C)c2n1,270.00,6.57,6.568636
CHEMBL1094408,COc1ccc(N2CCN(C)CC2)cc1Nc1ncc2c(n1)-c1c(c(C(N)...,219.00,6.66,6.659556
...,...,...,...,...
CHEMBL98554,O=C1Cc2c([nH]c3cc(Br)ccc23)-c2ccccc2N1,2697.74,5.57,5.569000
CHEMBL99037,O=C1Cc2c([nH]c3ccc(F)cc23)-c2ccccc2N1,1300.17,5.89,5.886000
CHEMBL99423,COc1cccc2c1NC(=O)Cc1c-2[nH]c2ccccc12,1000000.00,NaN,3.000000


In [ ]:
final_df = final_df.drop(['standard_value', 'pchembl_value'], axis = 1)
final_df

,canonical_smiles,log_ic50(pchembl)
molecule_chembl_id,,
CHEMBL100309,O=C1Cc2c([nH]c3ccc(C(F)(F)F)cc23)-c2cc(Br)ccc2N1,5.523000
CHEMBL100312,CC(C)(C)OC(=O)n1c2c(c3cc(Br)ccc31)CC(=O)Nc1ccc...,3.097000
CHEMBL1092509,O=C1Nc2c(Br)cccc2/C1=C1/Nc2ccccc2/C1=N\O,4.481486
CHEMBL1094304,CC[C@H](CO)Nc1nc(NCc2ccccc2O)c2ncn(C(C)C)c2n1,6.568636
CHEMBL1094408,COc1ccc(N2CCN(C)CC2)cc1Nc1ncc2c(n1)-c1c(c(C(N)...,6.659556
...,...,...
CHEMBL98554,O=C1Cc2c([nH]c3cc(Br)ccc23)-c2ccccc2N1,5.569000
CHEMBL99037,O=C1Cc2c([nH]c3ccc(F)cc23)-c2ccccc2N1,5.886000
CHEMBL99423,COc1cccc2c1NC(=O)Cc1c-2[nH]c2ccccc12,3.000000


In [ ]:
from rdkit import Chem

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

final_df["canonical_smiles"].apply(is_valid_smiles).sum() == final_df.shape[0]

np.True_

In [ ]:
final_df

,canonical_smiles,log_ic50(pchembl)
molecule_chembl_id,,
CHEMBL100309,O=C1Cc2c([nH]c3ccc(C(F)(F)F)cc23)-c2cc(Br)ccc2N1,5.523000
CHEMBL100312,CC(C)(C)OC(=O)n1c2c(c3cc(Br)ccc31)CC(=O)Nc1ccc...,3.097000
CHEMBL1092509,O=C1Nc2c(Br)cccc2/C1=C1/Nc2ccccc2/C1=N\O,4.481486
CHEMBL1094304,CC[C@H](CO)Nc1nc(NCc2ccccc2O)c2ncn(C(C)C)c2n1,6.568636
CHEMBL1094408,COc1ccc(N2CCN(C)CC2)cc1Nc1ncc2c(n1)-c1c(c(C(N)...,6.659556
...,...,...
CHEMBL98554,O=C1Cc2c([nH]c3cc(Br)ccc23)-c2ccccc2N1,5.569000
CHEMBL99037,O=C1Cc2c([nH]c3ccc(F)cc23)-c2ccccc2N1,5.886000
CHEMBL99423,COc1cccc2c1NC(=O)Cc1c-2[nH]c2ccccc12,3.000000


In [ ]:
final_df.to_csv('/content/drive/MyDrive/datacon2025/main/nlrp/processed_data.csv')

## Feature exctraction

#### Rdkit descriptors

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from tqdm.notebook import tqdm

In [ ]:
descriptor_list = Descriptors.descList

descriptor_names = [name for name, func in descriptor_list]

def calc_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    values = []
    for name, func in descriptor_list:
        try:
            val = func(mol)
        except:
            val = None
        values.append(val)
    return values

desc_values = final_df["canonical_smiles"].apply(calc_descriptors)
rdkit_df = pd.DataFrame(desc_values.tolist(), columns=descriptor_names)
rdkit_df

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.007274,13.007274,0.016808,-4.424338,0.546081,14.083333,395.178,385.098,393.992860,122,...,0,0,0,0,0,0,0,0,0,0
1,13.119549,13.119549,0.107024,-0.630714,0.526419,13.555556,427.298,408.146,426.057905,138,...,0,0,0,0,0,0,0,0,0,0
2,12.416573,12.416573,0.223744,-0.223744,0.384776,20.590909,356.179,346.099,354.995639,108,...,0,0,0,0,0,0,0,0,0,0
3,9.983450,9.983450,0.001973,-0.121399,0.482691,12.481481,370.457,344.249,370.211724,144,...,0,0,0,0,0,0,0,0,0,0
4,11.838012,11.838012,0.312795,-0.520220,0.604714,15.666667,448.531,420.307,448.233522,172,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,12.082851,12.082851,0.027795,0.027795,0.641661,13.550000,327.181,316.093,326.005475,98,...,0,0,0,0,0,0,0,0,0,0
1075,13.464348,13.464348,0.082073,-0.294222,0.642647,13.550000,266.275,255.187,266.085541,98,...,0,0,0,0,0,0,0,0,0,0
1076,12.237770,12.237770,0.023840,-0.023840,0.716991,13.285714,278.311,264.199,278.105528,104,...,0,0,0,0,0,0,0,0,0,0
1077,13.271006,13.271006,0.116601,0.116601,0.447254,13.370370,417.306,400.170,416.052425,132,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
nans = rdkit_df.isna().sum()
nans[nans > 0]

,0
BCUT2D_MWHI,4
BCUT2D_MWLOW,4
BCUT2D_CHGHI,4
BCUT2D_CHGLO,4
BCUT2D_LOGPHI,4
BCUT2D_LOGPLOW,4
BCUT2D_MRHI,4
BCUT2D_MRLOW,4


In [ ]:
nans = rdkit_df.isna().sum().to_dict()
nan_features = []
for key in nans:
  if nans[key] >0:
    nan_features.append(key)

rdkit_df = rdkit_df.fillna(rdkit_df.median())
(rdkit_df.isna().sum() > 0).sum()

np.int64(0)

In [ ]:
nans = rdkit_df.isna().sum()
nans[nans > 0]

,0


In [ ]:
zero_var_cols = [c for c in rdkit_df.columns if rdkit_df[c].nunique() <= 1]
zero_var_cols

['NumRadicalElectrons',
 'SMR_VSA8',
 'SlogP_VSA9',
 'fr_alkyl_carbamate',
 'fr_azide',
 'fr_barbitur',
 'fr_benzodiazepine',
 'fr_diazo',
 'fr_dihydropyridine',
 'fr_epoxide',
 'fr_isocyan',
 'fr_isothiocyan',
 'fr_lactam',
 'fr_phos_acid',
 'fr_phos_ester',
 'fr_prisulfonamd',
 'fr_quatN',
 'fr_thiocyan']

In [ ]:
rdkit_df = rdkit_df.drop(zero_var_cols, axis = 1)
len(rdkit_df.columns)

199

In [ ]:
import numpy as np

corr_matrix = rdkit_df.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

to_drop = [
    column for column in upper.columns if any(upper[column] > 0.95)
]

to_drop

['MaxEStateIndex',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'FpDensityMorgan3',
 'Chi0',
 'Chi0n',
 'Chi0v',
 'Chi1',
 'Chi1n',
 'Chi1v',
 'Chi2n',
 'Chi2v',
 'Chi3n',
 'Chi3v',
 'Chi4n',
 'Chi4v',
 'Kappa1',
 'Kappa2',
 'Kappa3',
 'LabuteASA',
 'HeavyAtomCount',
 'NOCount',
 'Phi',
 'MolMR',
 'fr_Al_OH_noTert',
 'fr_COO',
 'fr_COO2',
 'fr_C_O_noCOO',
 'fr_Nhpyrrole',
 'fr_amide',
 'fr_benzene',
 'fr_nitrile',
 'fr_nitro_arom',
 'fr_phenol_noOrthoHbond']

In [ ]:
rdkit_df = rdkit_df.drop(to_drop, axis = 1)
len(rdkit_df.columns)

164

In [ ]:
rdkit_df.to_csv('/content/drive/MyDrive/datacon2025/main/nlrp/rdkit_descriptors.csv')

#### MACCS fingerprints

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import numpy as np
from tqdm import tqdm

def compute_maccs_fingerprints(smiles_list):
    fps_array = []

    for smi in tqdm(smiles_list, desc="MACCS fingerprints"):
        mol = Chem.MolFromSmiles(smi)
        fp = MACCSkeys.GenMACCSKeys(mol)
        arr = np.zeros((fp.GetNumBits(),), dtype=int)
        Chem.DataStructs.ConvertToNumpyArray(fp, arr)
        fps_array.append(arr)

    fps_df = pd.DataFrame(fps_array, columns=[f"maccs_{i}" for i in range(167)])

    return fps_df


maccs_fps = compute_maccs_fingerprints(final_df["canonical_smiles"])

MACCS fingerprints: 100%|██████████| 1079/1079 [00:01<00:00, 630.13it/s]


In [ ]:
maccs_fps.shape

(1079, 167)

In [ ]:
maccs_fps.head()

,maccs_0,maccs_1,maccs_2,maccs_3,maccs_4,maccs_5,maccs_6,maccs_7,maccs_8,maccs_9,...,maccs_157,maccs_158,maccs_159,maccs_160,maccs_161,maccs_162,maccs_163,maccs_164,maccs_165,maccs_166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
maccs_fps.to_csv('/content/drive/MyDrive/datacon2025/main/nlrp/maccs_fingerprints.csv')

## Catboost

In [ ]:
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.4 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
all_features = pd.concat([rdkit_df, maccs_fps], axis = 1)

In [ ]:
X = all_features
y = final_df['log_ic50(pchembl)']
X.shape, y.shape

((1079, 331), (1079,))

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

rmse_scores = []
r2_scores = []
mae_scores = []

model_params = {
    "iterations": 1000,
    "learning_rate": 0.05,
    "depth": 6,
    "loss_function": "RMSE",
    "verbose": False,
    "random_seed": 42
}


fold = 1
for train_idx, val_idx in kf.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostRegressor(**model_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

    print(f"Fold {fold}: RMSE = {rmse:.4f}, MAE = {mae:.4f}, R2 = {r2:.4f}")
    fold += 1

print("\n--- Cross-validation results ---")
print(f"Mean RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")
print(f"Mean MAE: {np.mean(mae_scores):.4f} ± {np.std(mae_scores):.4f}")
print(f"Mean R2: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")


Fold 1: RMSE = 0.7058, MAE = 0.5249, R2 = 0.4508
Fold 2: RMSE = 0.7681, MAE = 0.5463, R2 = 0.4121
Fold 3: RMSE = 0.7195, MAE = 0.5004, R2 = 0.4676

--- Cross-validation results ---
Mean RMSE: 0.7311 ± 0.0267
Mean MAE: 0.5239 ± 0.0188
Mean R2: 0.4435 ± 0.0233


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.4 MB/s eta 0:00:00


In [ ]:
import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import numpy as np



def objective(trial):

    params = {
        "iterations": 500,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_float("random_strength", 0, 10),
        "rsm": trial.suggest_float("rsm", 0.5, 1.0),
        "loss_function": "RMSE",
        "early_stopping_rounds": 30,
        "verbose": False,
        "task_type": "CPU",
        "random_seed": 42
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    r2_scores = []

    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        train_pool = Pool(X_train, y_train)
        valid_pool = Pool(X_valid, y_valid)

        model = CatBoostRegressor(**params)
        model.fit(train_pool, eval_set=valid_pool)

        y_pred = model.predict(X_valid)
        r2 = r2_score(y_valid, y_pred)
        r2_scores.append(r2)

    return -np.mean(r2_scores)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Лучшие параметры:")
print(study.best_params)

print("Лучший средний R2:")
print(-study.best_value)


[I 2025-07-15 13:11:16,109] A new study created in memory with name: no-name-d792a3a9-6baf-43f5-9b6c-54168a321385
[I 2025-07-15 13:14:43,688] Trial 0 finished with value: -0.45332316568595477 and parameters: {'learning_rate': 0.031338100741972734, 'depth': 9, 'l2_leaf_reg': 8.683733323245203, 'bagging_temperature': 0.27265402591355836, 'border_count': 47, 'random_strength': 6.722290911787842, 'rsm': 0.9702175409834926}. Best is trial 0 with value: -0.45332316568595477.
[I 2025-07-15 13:16:22,961] Trial 1 finished with value: -0.503427585560645 and parameters: {'learning_rate': 0.048570347705017135, 'depth': 7, 'l2_leaf_reg': 4.638971219643663, 'bagging_temperature': 0.7248010306452692, 'border_count': 198, 'random_strength': 8.789513962027733, 'rsm': 0.8353918347766416}. Best is trial 1 with value: -0.503427585560645.
[I 2025-07-15 13:16:42,551] Trial 2 finished with value: -0.4971842583835911 and parameters: {'learning_rate': 0.06203442462795378, 'depth': 5, 'l2_leaf_reg': 3.764980393

Лучшие параметры:
{'learning_rate': 0.09568616606408391, 'depth': 6, 'l2_leaf_reg': 5.058050236211532, 'bagging_temperature': 0.384419865053069, 'border_count': 115, 'random_strength': 4.474511785940289, 'rsm': 0.7418307758806199}
Лучший средний R2:
0.512901349055977


In [ ]:
import json
json.dump(
    study.best_params,
    open('/content/drive/MyDrive/datacon2025/main/nlrp/catboost_best_params.json', 'w')
)

In [105]:
model = CatBoostRegressor(**study.best_params)
model.fit(X, y)

0:	learn: 0.9683137	total: 99ms	remaining: 1m 38s
1:	learn: 0.9520049	total: 125ms	remaining: 1m 2s
2:	learn: 0.9381384	total: 168ms	remaining: 55.8s
3:	learn: 0.9238638	total: 190ms	remaining: 47.4s
4:	learn: 0.9093163	total: 202ms	remaining: 40.3s
5:	learn: 0.8967811	total: 215ms	remaining: 35.5s
6:	learn: 0.8878394	total: 232ms	remaining: 32.9s
7:	learn: 0.8827252	total: 244ms	remaining: 30.2s
8:	learn: 0.8759507	total: 256ms	remaining: 28.2s
9:	learn: 0.8696683	total: 268ms	remaining: 26.5s
10:	learn: 0.8622780	total: 289ms	remaining: 25.9s
11:	learn: 0.8545204	total: 302ms	remaining: 24.9s
12:	learn: 0.8466438	total: 314ms	remaining: 23.9s
13:	learn: 0.8415643	total: 327ms	remaining: 23.1s
14:	learn: 0.8354807	total: 340ms	remaining: 22.3s
15:	learn: 0.8305719	total: 352ms	remaining: 21.6s
16:	learn: 0.8236447	total: 364ms	remaining: 21.1s
17:	learn: 0.8159491	total: 376ms	remaining: 20.5s
18:	learn: 0.8100966	total: 388ms	remaining: 20s
19:	learn: 0.8041068	total: 400ms	remaining

In [108]:
model.save_model("/content/drive/MyDrive/datacon2025/main/nlrp/catboost_nlrp_final.json", format="json")

## Checking candidates

In [109]:
from rdkit import Chem
from rdkit.Contrib.SA_Score import sascorer
from rdkit.Chem import FilterCatalog
from rdkit.Chem.FilterCatalog import FilterCatalogParams
from rdkit.Chem import Descriptors
from IPython.display import display
from rdkit.Chem import Draw

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog.FilterCatalog(params)
#df = concat_mols()


# Функция проверки по 4 критериям
def checker(df):
  dfs=df.copy()
  indexes=[]
  for index, row in dfs.iterrows():

    mol = Chem.MolFromSmiles(dfs['smiles'][index])
    MW = Descriptors.MolWt(mol)
    HBA = Descriptors.NOCount(mol)
    HBD = Descriptors.NHOHCount(mol)
    LogP = Descriptors.MolLogP(mol)
    print(Chem.QED.qed(mol))
    if (dfs['pValue'][index] < 3) or (Chem.QED.qed(mol) < 0.5) or ((sascorer.calculateScore(mol) < 6) and (sascorer.calculateScore(mol) > 2)) or (catalog.HasMatch(mol) == True) or ((MW <= 500 and LogP <= 5 and HBD <= 5 and HBA <= 10) == False):
      indexes.append(index)
  df = dfs.drop(indexes)
  #print(df)

  # for index, row in df.iterrows():
  #   mol = Chem.MolFromSmiles(dfs['Smiles'][index])
  #   img = Draw.MolToImage(mol, size=(300,300))
  #   display(img)

  return df
  # df.to_csv('/content/selected_hits.csv',
  #             #sep = ';',
  #

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


In [112]:
cdk = pd.read_csv('/content/drive/MyDrive/datacon2025/main/potential_candidates/hash_ligand_mapping_CDK5R1.csv')
cdk = cdk[cdk["smiles"].apply(is_valid_smiles)]

[19:34:00] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 17 18 19
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 12 14 16
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 13 15 16 17 19 20 21
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 15 17 18 19 20 21 22
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 15 18 19 20 21 22 23
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 16 17 19 20 21 22 23
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 23
[19:34:00] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19 20 21 22 23 24 25


In [113]:
values = cdk["smiles"].apply(calc_descriptors)
candidates_rdkit = pd.DataFrame(values.tolist(), columns=descriptor_names)
candidates_rdkit

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.111270,13.111270,0.130823,-0.438488,0.280555,12.181818,493.371,479.259,492.010219,162,...,0,0,0,0,0,0,0,1,0,1
1,13.062977,13.062977,0.012268,-0.606622,0.398353,11.129032,448.265,437.177,447.017747,150,...,0,0,0,0,0,0,0,0,0,0
2,12.108414,12.108414,0.101878,-0.101878,0.694369,10.208333,383.249,368.129,382.042923,120,...,0,0,0,0,0,0,0,0,0,0
3,13.119592,13.119592,0.068113,-4.534122,0.531164,20.500000,363.299,351.203,363.094309,134,...,0,0,0,0,0,0,0,0,0,0
4,12.846598,12.846598,0.038798,-0.038798,0.556787,14.392857,439.353,416.169,438.094290,144,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,11.957738,11.957738,0.134426,-0.134426,0.726467,10.722222,256.692,247.620,256.040341,88,...,0,0,0,0,0,0,0,0,0,0
498,6.290646,6.290646,0.596192,0.596192,0.452989,10.807692,377.900,361.772,377.075346,128,...,0,0,0,0,0,0,0,0,0,0
499,11.279237,11.279237,0.166139,-0.486623,0.498640,10.814815,355.393,338.257,355.120843,132,...,0,0,0,0,0,0,0,0,0,0
500,13.496801,13.496801,0.262401,-4.781396,0.814787,21.785714,392.356,376.228,392.126024,146,...,0,0,0,0,0,0,0,0,0,0


In [114]:
candidates_rdkit = candidates_rdkit[rdkit_df.columns]

In [115]:
maccs_fps = compute_maccs_fingerprints(cdk["smiles"])
candidates_features = pd.concat([candidates_rdkit, maccs_fps], axis = 1)
pvalues = model.predict(candidates_features)
pvalues

MACCS fingerprints: 100%|██████████| 502/502 [00:01<00:00, 383.14it/s]


array([5.84512878, 6.00380659, 6.83812899, 6.82361885, 5.50283562,
       5.75174693, 6.08312543, 6.16181202, 7.40066796, 5.65457951,
       5.61966936, 5.86310246, 4.80933532, 5.63855581, 6.33937248,
       4.89663868, 6.18879306, 5.52406914, 5.93213478, 4.97698418,
       5.78869779, 5.57741186, 6.53202079, 5.08440485, 6.54432969,
       6.47688687, 6.09019173, 7.17327171, 4.84789211, 5.10276657,
       7.11083663, 6.7566441 , 5.02332201, 7.21055424, 6.52248937,
       6.50660233, 5.51851185, 5.94437629, 4.26237051, 5.3159289 ,
       5.05123498, 6.84519334, 4.53159134, 6.95299081, 6.37788099,
       5.96727927, 5.84170196, 6.08715795, 6.68328285, 5.34604008,
       6.58629126, 6.44518807, 6.46991648, 6.18775606, 6.5144216 ,
       5.41976869, 7.18981087, 6.41014122, 7.2222228 , 5.55898296,
       5.82499074, 5.35969014, 6.44964926, 6.75816437, 5.43813753,
       7.03715758, 5.20910298, 6.46024297, 5.95126443, 6.55653939,
       7.50142254, 6.70043625, 6.11490923, 6.20701606, 5.93644

In [116]:
cdk['pValue'] = pvalues

#### Checking on blood brain barrier permeability

In [117]:
!git clone https://github.com/12rajnish/DeePred-BBB

Cloning into 'DeePred-BBB'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 61 (delta 5), reused 2 (delta 2), pack-reused 49 (from 1)
Receiving objects: 100% (61/61), 67.82 MiB | 26.04 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [121]:
cdk[['smiles','file_name']].to_csv(
    '/content/DeePred-BBB/smiles.smi',
    sep = ' ',
    header=False,
    index = False
    )

In [122]:
!python /content/DeePred-BBB/DeePred-BBB_Script.py /content/DeePred-BBB/

2025-07-15 19:36:19.543575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752608179.934135  190279 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752608180.044013  190279 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-15 19:36:20.917832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Processing 656eaab3e4c5451ff66454b56e0748e9dc8741f9 in smiles.smi (1/502). 
Processing 0d7e800964fadef6c21c58be35665f1ddb277d

In [125]:
preds = pd.read_csv('/content/DeePred-BBB_predictions.csv')
preds = preds[(preds['Predicted_class'] == 1)]
filtered_df = cdk[cdk["file_name"].isin(preds['Name'])]

In [126]:
res = checker(filtered_df)

0.6943688789730827
0.7041628351627279
0.5988641374864287
0.5363189174286709
0.22461806730011027
0.38473679347733686
0.90846160463585
0.6630211653669337
0.6835311304559605
0.7398865389441668
0.8455620580319114
0.44163697502276394
0.452741073111679
0.7325159011436986
0.7126623361368025
0.6591975140279975
0.5068429224790628
0.5796410050409321
0.4980690597062398
0.39568280831921193
0.7497210373039224
0.6171852800635086
0.7057559785445487
0.537531367201701
0.8051984585166931
0.7459495493518656
0.7491463059697292
0.48971597574113235
0.27987265747628604
0.41438558556211735
0.5168725979675981
0.34297263045705784
0.7902807192805471
0.5184906920480963
0.5650035583534408
0.8698803112461235
0.8381257387699341
0.6952391199272709
0.7540955385597045
0.7904963744612046
0.5823312025535552
0.7293284145495937
0.46627840915281354
0.8249000810846697
0.6228424907672464
0.8133711972993234
0.6721474083326914
0.7755510502859606
0.717593610281092
0.8924479303602874
0.6951462641633915
0.3206753130154673
0.570568

In [127]:
res.shape

(10, 3)

In [128]:
res.to_csv('/content/drive/MyDrive/datacon2025/main/cdk5/approved_molecules.csv')

In [129]:
res['pValue'].mean(), res['pValue'].max(), res['pValue'].min()

(np.float64(5.773634263337485), 6.6256170884833745, 4.809411991116509)